# Complete Email System with Advanced Features

This notebook demonstrates a comprehensive email system with advanced network topology, multiple protocols (SMTP and POP3), and professional system monitoring using built-in `.show()` methods.

## Overview

This advanced tutorial showcases enterprise-level email system capabilities:

| **Component** | **Type** | **Purpose** | **Advanced Features** |
|:-------------:|:--------:|:-----------:|:---------------------:|
| **Network Switch** | Infrastructure | Central connectivity | Multi-port switching, proper topology |
| **SMTP Server** | Service | Email sending/relay | Mailbox management, session handling |
| **POP3 Server** | Service | Email retrieval | Authentication, message retrieval |
| **Multiple Clients** | Applications | User interfaces | Distributed user access |
| **System Monitoring** | Tools | Health checking | Comprehensive logging, status display |

## Learning Objectives

1. **Advanced Networking**: Switch-based topology with multiple clients
2. **Multi-Protocol Support**: Both SMTP and POP3 services
3. **System Monitoring**: Professional display methods and logging
4. **Email Workflows**: Complete send/receive cycles
5. **Troubleshooting**: Advanced debugging and health checking
6. **Enterprise Patterns**: Scalable, production-like configurations

## Table of Contents

- [Environment Setup](#environment-setup)
- [Advanced Network Architecture](#advanced-network-architecture)
- [Multi-Service Installation](#multi-service-installation)
- [System Status Monitoring](#system-status-monitoring)
- [Email Communication Workflows](#email-communication-workflows)
- [Advanced Display Methods](#advanced-display-methods)
- [POP3 Email Retrieval](#pop3-email-retrieval)
- [Comprehensive System Analysis](#comprehensive-system-analysis)
- [Best Practices and Cleanup](#best-practices-and-cleanup)

In [1]:
!primaite setup

2025-09-26 22:14:51,852: Performing the PrimAITE first-time setup...
2025-09-26 22:14:51,852: Building the PrimAITE app directories...
2025-09-26 22:14:51,852: Building primaite_config.yaml...
2025-09-26 22:14:51,852: Rebuilding the demo notebooks...
2025-09-26 22:14:51,864: Rebuilding the example notebooks...
2025-09-26 22:14:51,865: PrimAITE setup complete!


In [2]:
import primaite
from primaite import PRIMAITE_CONFIG
from primaite.simulator.network.container import Network
from primaite.simulator.network.hardware.nodes.host.computer import Computer
from primaite.simulator.network.hardware.nodes.network.switch import Switch
from primaite_mail.simulator.software.smtp_server import SMTPServer
from primaite_mail.simulator.software.pop3_server import POP3Server
from primaite_mail.simulator.software.email_client import EmailClient
from primaite_mail.simulator.network.protocols.smtp import EmailMessage
import logging

# Store original config to restore later
from primaite import PRIMAITE_CONFIG
original_dev_mode = PRIMAITE_CONFIG["developer_mode"]["enabled"]
original_sys_logs = PRIMAITE_CONFIG["developer_mode"]["output_sys_logs"]
original_agent_logs = PRIMAITE_CONFIG["developer_mode"]["output_agent_logs"]

# Enable developer mode and system logs for debugging
PRIMAITE_CONFIG["developer_mode"]["enabled"] = True
PRIMAITE_CONFIG["developer_mode"]["output_sys_logs"] = True
PRIMAITE_CONFIG["developer_mode"]["output_agent_logs"] = True

print("Developer mode enabled for enhanced debugging")
print(f"System logs enabled: {PRIMAITE_CONFIG['developer_mode']['output_sys_logs']}")

Developer mode enabled for enhanced debugging
System logs enabled: True


## Network Setup with Switch

Create a proper network topology using a switch to connect all devices.

In [3]:
def create_email_network() -> Network:
    """Create a complete email network infrastructure with proper topology."""
    network = Network()
    
    # Create a switch to connect all devices
    switch = Switch.from_config({
        "type": "switch",
        "hostname": "main_switch",
        "num_ports": 8,
        "start_up_duration": 0,
    })
    switch.power_on()
    network.add_node(switch)
    
    # Create dedicated mail server
    mail_server = Computer.from_config({
        "type": "computer",
        "hostname": "mail_server",
        "ip_address": "192.168.1.10",
        "subnet_mask": "255.255.255.0",
        "start_up_duration": 0,
    })
    mail_server.power_on()
    network.add_node(mail_server)
    
    # Connect mail server to switch
    network.connect(mail_server.network_interface[1], switch.network_interface[1])
    
    # Create client computers
    clients = []
    for i, name in enumerate(["alice_pc", "bob_pc", "charlie_pc"], 1):
        client = Computer.from_config({
            "type": "computer",
            "hostname": name,
            "ip_address": f"192.168.1.{20 + i}",
            "subnet_mask": "255.255.255.0",
            "start_up_duration": 0,
        })
        client.power_on()
        network.add_node(client)
        clients.append(client)
        
        # Connect each client to the switch (using different switch ports)
        network.connect(client.network_interface[1], switch.network_interface[i + 1])
    
    return network

# Create the network
print("Creating email network with switch...")
network = create_email_network()
mail_server = network.get_node_by_hostname("mail_server")
alice_pc = network.get_node_by_hostname("alice_pc")
bob_pc = network.get_node_by_hostname("bob_pc")
charlie_pc = network.get_node_by_hostname("charlie_pc")
switch = network.get_node_by_hostname("main_switch")

print(f"Network created with {len(network.nodes)} nodes")
print(f"Switch: {switch.config.hostname}")
print(f"Mail server: {mail_server.config.hostname} ({mail_server.config.ip_address})")
for client in [alice_pc, bob_pc, charlie_pc]:
    print(f"Client: {client.config.hostname} ({client.config.ip_address})")

# Test basic connectivity
print("\nTesting connectivity...")
# Note: You may see warnings about network interfaces during startup - this is normal
ping_result = alice_pc.ping(mail_server.network_interface[1].ip_address)
print(f"Alice -> Mail Server: {'SUCCESS' if ping_result else 'FAILED'}")

Creating email network with switch...
Network created with 5 nodes
Switch: main_switch
Mail server: mail_server (192.168.1.10)
Client: alice_pc (192.168.1.21)
Client: bob_pc (192.168.1.22)
Client: charlie_pc (192.168.1.23)

Testing connectivity...
Alice -> Mail Server: SUCCESS


## Install Email Services

In [4]:
# Install email servers on mail server
print("Installing email servers...")
mail_server.software_manager.install(SMTPServer)
mail_server.software_manager.install(POP3Server)

smtp_server = mail_server.software_manager.software.get("smtp-server")
pop3_server = mail_server.software_manager.software.get("pop3-server")

print(f"SMTP Server: {smtp_server.name} (Status: {smtp_server.operating_state.name}, Port: {smtp_server.port})")
print(f"POP3 Server: {pop3_server.name} (Status: {pop3_server.operating_state.name}, Port: {pop3_server.port})")

# Install email clients on user machines
print("\nInstalling email clients...")
clients = {}
for pc, username in [(alice_pc, "alice"), (bob_pc, "bob"), (charlie_pc, "charlie")]:
    pc.software_manager.install(EmailClient)
    client = pc.software_manager.software.get("email-client")
    # Configure client with server details
    client.config.username = f"{username}@company.com"
    client.config.default_smtp_server = str(mail_server.config.ip_address)
    client.config.default_pop3_server = str(mail_server.config.ip_address)
    # IMPORTANT: Start the email client after installation
    client.run()
    clients[username] = client
print(f"{username.title()}'s email client: {client.name} (Status: {client.operating_state.name})")

# Create mailboxes for users
print("\nCreating user mailboxes...")
users = ["alice", "bob", "charlie"]
for user in users:
    smtp_success = smtp_server.mailbox_manager.create_mailbox(user)
    print(f"Mailbox for {user}: {'Created' if smtp_success else 'Already exists'}")

# Share mailbox manager between SMTP and POP3 servers
pop3_server.mailbox_manager = smtp_server.mailbox_manager

# IMPORTANT: Configure POP3 authentication for testing
pop3_server.config.require_auth = False  # Disable auth for easier testing

print(f"\nTotal mailboxes: {len(smtp_server.mailbox_manager.mailboxes)}")

Installing email servers...
SMTP Server: smtp-server (Status: RUNNING, Port: 25)
POP3 Server: pop3-server (Status: RUNNING, Port: 110)

Installing email clients...
Charlie's email client: email-client (Status: RUNNING)

Creating user mailboxes...
Mailbox for alice: Created
Mailbox for bob: Created
Mailbox for charlie: Created

Total mailboxes: 3


## Display Initial System Status

Use the built-in `.show()` methods to display clean, tabular system information.

In [ ]:
print("=== INITIAL SYSTEM STATUS ===")
print("\n1. SMTP Server Status:")
smtp_server.show()

print("\n2. POP3 Server Status:")
pop3_server.show()

print("\n3. Mailbox Manager Status:")
smtp_server.mailbox_manager.show()

print("\n5. System Logs (Mail Server):")
mail_server.sys_log.show(last_n=10)

=== INITIAL SYSTEM STATUS ===

1. SMTP Server Status:
+-------------------------------------+
|   SMTP Server Status (mail_server)  |
+-----------------------+-------------+
| Property              | Value       |
+-----------------------+-------------+
| Service Name          | smtp-server |
| Operating State       | RUNNING     |
| Health State          | GOOD        |
| Port                  | 25          |
| Protocol              | tcp         |
| Active Sessions       | 0           |
| Total Mailboxes       | 3           |
| Max Attachment Size   | 25 MB       |
| Max Total Size        | 50 MB       |
| Malware Scanning      | Enabled     |
| Quarantine Suspicious | Enabled     |
| Security Logging      | Enabled     |
| Blocked Senders       | 0           |
| Blocked IPs           | 0           |
| Security Events       | 0           |
+-----------------------+-------------+
+-------------------------------------------------------------------------+
|                      Mailbox

## Test Email Sending

In [6]:
# Alice sends an email to Bob and Charlie
email1 = EmailMessage(
    sender="alice@company.com",
    recipients=["bob@company.com", "charlie@company.com"],
    subject="Team Meeting Tomorrow",
    body="Hi everyone, don't forget about our team meeting tomorrow at 10 AM in the conference room."
)

print("=== ALICE SENDING EMAIL ===")
print(f"From: {email1.sender}")
print(f"To: {', '.join(email1.recipients)}")
print(f"Subject: {email1.subject}")
print(f"Target server: {mail_server.config.ip_address}")

success = clients["alice"].send_email(email1, mail_server.config.ip_address)
print(f"\nAlice's email result: {'Successful' if success else 'Failed'}")

# Show updated system status after email
print("\n=== SYSTEM STATUS AFTER ALICE'S EMAIL ===")
print("\nSMTP Server Status:")
smtp_server.show()

print("\nMailbox Contents:")
smtp_server.mailbox_manager.show(show_messages=True)

print("\nAlice's Client Logs:")
alice_pc.sys_log.show(last_n=8)

print("\nMail Server Logs:")
mail_server.sys_log.show(last_n=8)

=== ALICE SENDING EMAIL ===
From: alice@company.com
To: bob@company.com, charlie@company.com
Subject: Team Meeting Tomorrow
Target server: 192.168.1.10

Alice's email result: Successful

=== SYSTEM STATUS AFTER ALICE'S EMAIL ===

SMTP Server Status:
+-------------------------------------+
|   SMTP Server Status (mail_server)  |
+-----------------------+-------------+
| Property              | Value       |
+-----------------------+-------------+
| Service Name          | smtp-server |
| Operating State       | RUNNING     |
| Health State          | GOOD        |
| Port                  | 25          |
| Protocol              | tcp         |
| Active Sessions       | 0           |
| Total Mailboxes       | 3           |
| Max Attachment Size   | 25 MB       |
| Max Total Size        | 50 MB       |
| Malware Scanning      | Enabled     |
| Quarantine Suspicious | Enabled     |
| Security Logging      | Enabled     |
| Blocked Senders       | 0           |
| Blocked IPs           | 0   

In [7]:
# Bob replies to Alice
email2 = EmailMessage(
    sender="bob@company.com",
    recipients=["alice@company.com"],
    subject="Re: Team Meeting Tomorrow",
    body="Thanks for the reminder, Alice! I'll be there."
)

print("=== BOB SENDING REPLY ===")
print(f"From: {email2.sender}")
print(f"To: {', '.join(email2.recipients)}")
print(f"Subject: {email2.subject}")

success = clients["bob"].send_email(email2, mail_server.config.ip_address)
print(f"\nBob's reply result: {'Successful' if success else 'Failed'}")

# Charlie also sends a reply
email3 = EmailMessage(
    sender="charlie@company.com",
    recipients=["alice@company.com"],
    subject="Re: Team Meeting Tomorrow",
    body="Count me in! Should I bring the quarterly reports?"
)

print("\n=== CHARLIE SENDING REPLY ===")
print(f"From: {email3.sender}")
print(f"To: {', '.join(email3.recipients)}")
print(f"Subject: {email3.subject}")

success = clients["charlie"].send_email(email3, mail_server.config.ip_address)
print(f"\nCharlie's reply result: {'Successful' if success else 'Failed'}")

=== BOB SENDING REPLY ===
From: bob@company.com
To: alice@company.com
Subject: Re: Team Meeting Tomorrow

Bob's reply result: Successful

=== CHARLIE SENDING REPLY ===
From: charlie@company.com
To: alice@company.com
Subject: Re: Team Meeting Tomorrow

Charlie's reply result: Successful


## Demonstrate New Mailbox Display Methods

Show the new `.show_mailbox()` and `.show_message()` methods that provide clean, tabular display of mailbox contents.

In [8]:
print("=== NEW MAILBOX DISPLAY METHODS ===")

print("\n1. SMTP Server - All Mailboxes:")
smtp_server.show_mailbox()

print("\n2. SMTP Server - Alice's Mailbox Only:")
smtp_server.show_mailbox("alice")

print("\n3. POP3 Server - Bob's Mailbox (POP3 View):")
pop3_server.show_mailbox("bob")

# Show detailed message content
alice_mailbox = smtp_server.mailbox_manager.get_mailbox("alice")
if alice_mailbox and len(alice_mailbox.get_messages()) > 0:
    print("\n4. Detailed Message View (Alice's first message):")
    smtp_server.show_message("alice", 1)

bob_mailbox = smtp_server.mailbox_manager.get_mailbox("bob")
if bob_mailbox and len(bob_mailbox.get_messages()) > 0:
    print("\n5. POP3 Message View (Bob's first message):")
    pop3_server.show_message("bob", 1)

print("\n6. Email Client Mailbox View:")
clients["alice"].show_mailbox()

print("\n7. Markdown Format Example:")
smtp_server.show_mailbox("charlie", markdown=True)

=== NEW MAILBOX DISPLAY METHODS ===

1. SMTP Server - All Mailboxes:

+---------------------------------------------------------------------------------------------------------------------+
|                                        alice's Mailbox Contents (2 messages)                                        |
+---+---------------------+-------------------+---------------------------+---------------------+-------------+-------+
| # | From                | To                | Subject                   | Timestamp           | Attachments | Size  |
+---+---------------------+-------------------+---------------------------+---------------------+-------------+-------+
| 1 | bob@company.com     | alice@company.com | Re: Team Meeting Tomorrow | 2025-09-26T22:14:53 | 0           | 103 B |
| 2 | charlie@company.com | alice@company.com | Re: Team Meeting Tomorrow | 2025-09-26T22:14:53 | 0           | 111 B |
+---+---------------------+-------------------+---------------------------+---------------

## Test POP3 Email Retrieval

In [9]:
# Bob retrieves his emails via POP3
print("=== BOB RETRIEVING EMAILS VIA POP3 ===")
print(f"POP3 server: {mail_server.config.ip_address}")
print(f"Username: bob")

emails = clients["bob"].retrieve_emails_pop3(
    pop3_server_ip=mail_server.config.ip_address,
    username="bob",
    password="password123"
)

print(f"\nPOP3 retrieval result: {'Success' if emails is not None else 'Failed'}")
if emails is not None:
    print(emails)
    print(f"Retrieved {len(emails)} emails")
    for i, email in enumerate(emails, 1):
        print(f"  {i}. Subject: {email.subject}")
        print(f"     From: {email.sender}")

# Show POP3 server status after retrieval attempt
print("\n=== POP3 SERVER STATUS AFTER RETRIEVAL ===")
pop3_server.show()

print("\nBob's Client Logs:")
bob_pc.sys_log.show(last_n=20)

=== BOB RETRIEVING EMAILS VIA POP3 ===
POP3 server: 192.168.1.10
Username: bob

POP3 retrieval result: Failed

=== POP3 SERVER STATUS AFTER RETRIEVAL ===
+---------------------------------------+
|    POP3 Server Status (mail_server)   |
+-------------------------+-------------+
| Property                | Value       |
+-------------------------+-------------+
| Service Name            | pop3-server |
| Operating State         | RUNNING     |
| Health State            | GOOD        |
| Port                    | 110         |
| Protocol                | tcp         |
| Active Sessions         | 0           |
| Total Mailboxes         | 3           |
| Authentication Required | False       |
+-------------------------+-------------+

Bob's Client Logs:
+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                        bob_

## Final System Status and Logs

In [10]:
print("=== FINAL COMPREHENSIVE SYSTEM STATUS ===")

# Network summary
total_messages = sum(len(mailbox.get_messages()) for mailbox in smtp_server.mailbox_manager.mailboxes.values())
print(f"\nNetwork Summary:")
print(f"- Total nodes: {len(network.nodes)} (1 switch + 1 mail server + 3 clients)")
print(f"- Total links: {len(network.links)}")
print(f"- Total messages in system: {total_messages}")
print(f"- Active SMTP sessions: {len(smtp_server.active_sessions)}")
print(f"- Active POP3 sessions: {len(pop3_server.active_sessions)}")

# Final status displays
print("\n=== FINAL SERVER STATUS ===")
print("\n1. SMTP Server:")
smtp_server.show()

print("\n2. POP3 Server:")
pop3_server.show()

print("\n3. Complete Mailbox System:")
smtp_server.mailbox_manager.show(show_messages=True)

print("\n4. All Email Clients:")
for username, client in clients.items():
    print(f"\n{username.title()}'s Email Client:")
    client.show()

# System logs for debugging
print("\n=== SYSTEM LOGS ===")
print("\nMail Server System Logs:")
mail_server.sys_log.show(last_n=15)

print("\nAlice PC System Logs:")
alice_pc.sys_log.show(last_n=10)

print("\nBob PC System Logs:")
bob_pc.sys_log.show(last_n=10)

print("\nCharlie PC System Logs:")
charlie_pc.sys_log.show(last_n=10)

=== FINAL COMPREHENSIVE SYSTEM STATUS ===

Network Summary:
- Total nodes: 5 (1 switch + 1 mail server + 3 clients)
- Total links: 4
- Total messages in system: 4
- Active SMTP sessions: 0
- Active POP3 sessions: 0

=== FINAL SERVER STATUS ===

1. SMTP Server:
+-------------------------------------+
|   SMTP Server Status (mail_server)  |
+-----------------------+-------------+
| Property              | Value       |
+-----------------------+-------------+
| Service Name          | smtp-server |
| Operating State       | RUNNING     |
| Health State          | GOOD        |
| Port                  | 25          |
| Protocol              | tcp         |
| Active Sessions       | 0           |
| Total Mailboxes       | 3           |
| Max Attachment Size   | 25 MB       |
| Max Total Size        | 50 MB       |
| Malware Scanning      | Enabled     |
| Quarantine Suspicious | Enabled     |
| Security Logging      | Enabled     |
| Blocked Senders       | 0           |
| Blocked IPs      

## Cleanup and Configuration Restore

In [11]:
# Restore original configuration
print("=== RESTORING ORIGINAL CONFIGURATION ===")
PRIMAITE_CONFIG["developer_mode"]["enabled"] = original_dev_mode
PRIMAITE_CONFIG["developer_mode"]["output_sys_logs"] = original_sys_logs
print(f"Developer mode restored to: {original_dev_mode}")
print(f"System logs restored to: {original_sys_logs}")
print("\nEmail system demonstration completed successfully!")
print("\n=== AVAILABLE DISPLAY METHODS ===")
print("- smtp_server.show() - Server status and mailbox summary")
print("- smtp_server.show_mailbox(username) - Mailbox contents")
print("- smtp_server.show_message(username, msg_num) - Message details")
print("- pop3_server.show() - POP3 server status")
print("- pop3_server.show_mailbox(username) - POP3 mailbox view")
print("- email_client.show() - Client configuration")
print("- mailbox_manager.show(show_messages=True) - All mailboxes")
print("- node.sys_log.show(last_n=10) - System logs")

=== RESTORING ORIGINAL CONFIGURATION ===
Developer mode restored to: True
System logs restored to: False

Email system demonstration completed successfully!

=== AVAILABLE DISPLAY METHODS ===
- smtp_server.show() - Server status and mailbox summary
- smtp_server.show_mailbox(username) - Mailbox contents
- smtp_server.show_message(username, msg_num) - Message details
- pop3_server.show() - POP3 server status
- pop3_server.show_mailbox(username) - POP3 mailbox view
- email_client.show() - Client configuration
- mailbox_manager.show(show_messages=True) - All mailboxes
- node.sys_log.show(last_n=10) - System logs


In [12]:
# Returning to the original developer mode configuration.
PRIMAITE_CONFIG["developer_mode"]["enabled"] = original_dev_mode
PRIMAITE_CONFIG["developer_mode"]["output_sys_logs"] = original_sys_logs
PRIMAITE_CONFIG["developer_mode"]["output_agent_logs"] = original_agent_logs